In [1]:
!pip install torch torchaudio transformers datasets vector-quantize-pytorch torchcodec jiwer wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103

In [2]:
!wandb login bcf87665e6100865897ebf8c1c0bdcc8a9b72283

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [3]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2ForCTC
from datasets import load_from_disk, load_dataset
from tqdm import tqdm
from jiwer import wer
from torch.utils.data import DataLoader
import wandb
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.utils.rnn import pad_sequence
import random
import numpy as np

2025-09-29 05:42:46.344085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759124566.526527      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759124566.581677      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
#1️⃣ Set a fixed seed number
SEED = 42

# 2️⃣ Python random seed
random.seed(SEED)

# 3️⃣ NumPy random seed
np.random.seed(SEED)

# 4️⃣ PyTorch random seed
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # if using multi-GPU

# 5️⃣ For cudnn backend reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
# from google.colab import drive
# drive.mount("/content/drive")

In [6]:
wandb.init(
    project = "Wav2Vec2 finetuning on low resourse language",
    name = "Without RVQ - Exp001 - basic",    
)

wandb: Currently logged in as: rasarathathsarana63 (rasarathathsarana63-university-of-moratuwa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250929_054303-n7vp2udy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Without RVQ - Exp001 - basic
wandb: ⭐️ View project at https://wandb.ai/rasarathathsarana63-university-of-moratuwa/Wav2Vec2%20finetuning%20on%20low%20resourse%20language
wandb: 🚀 View run at https://wandb.ai/rasarathathsarana63-university-of-moratuwa/Wav2Vec2%20finetuning%20on%20low%20resourse%20language/runs/n7vp2udy


In [7]:
MODEL_NAME = "facebook/wav2vec2-base"
LEARNING_RATE = 1e-4
WARMUP_RATIO = 0.2
HOLD_RATIO = 0.2
DECAY_RATIO = 0.6
BATCH_SIZE = 8
EPOCHS = 30
GRADIENT_ACCUMULATION_STEPS = 8
LOGGING_STEPS = 10

In [8]:
def tri_stage_lr_lambda(current_step: int, 
                        total_steps: int,
                        warmup_ratio: float = 0.1, 
                        hold_ratio: float = 0.1, 
                        decay_ratio: float = 0.8):
    """
    Tri-stage learning rate schedule using ratios instead of raw steps.
    
    Args:
        current_step (int): Current step.
        total_steps (int): Total number of training steps.
        warmup_ratio (float): Fraction of steps for warmup.
        hold_ratio (float): Fraction of steps for hold.
        decay_ratio (float): Fraction of steps for decay.
    """
    # Convert ratios -> absolute steps
    warmup_steps = int(total_steps * warmup_ratio)
    hold_steps = int(total_steps * hold_ratio)
    decay_steps = int(total_steps * decay_ratio)

    if current_step < warmup_steps:
        # Linear warmup
        return float(current_step) / float(max(1, warmup_steps))
    elif current_step < warmup_steps + hold_steps:
        # Hold base LR
        return 1.0
    elif current_step < warmup_steps + hold_steps + decay_steps:
        # Linear decay
        decay_progress = (current_step - warmup_steps - hold_steps) / float(decay_steps)
        return max(0.0, 1.0 - decay_progress)
    else:
        # After decay, keep LR at 0
        return 0.0


In [9]:
# ------------------------
# 1️⃣ Load pre-trained Wav2Vec2
# ------------------------
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
wav2vec_model = Wav2Vec2Model.from_pretrained(MODEL_NAME)

# Freeze original model to save compute
for param in wav2vec_model.parameters():
    param.requires_grad = False


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

In [10]:
# ------------------------
# 3️⃣ Define downstream CTC model
# ------------------------
class Wav2Vec2WithRVQ(nn.Module):
    def __init__(self, wav2vec_model, vocab_size):
        super().__init__()
        self.wav2vec = wav2vec_model
        self.classifier = nn.Linear(wav2vec_model.config.hidden_size, vocab_size)

    def forward(self, input_values):
        outputs = self.wav2vec(input_values).last_hidden_state  # [B, T, H]
        logits = self.classifier(outputs)                      # [B, T, vocab]
        return logits

# Vocabulary (use processor vocab size)
vocab_size = len(processor.tokenizer)
model = Wav2Vec2WithRVQ(wav2vec_model, vocab_size)


In [11]:
# ------------------------
# 4️⃣ Prepare dataset
# ------------------------
train_dataset = load_from_disk("/kaggle/input/librispeech-10h/dataset_10h")
valid_dataset = load_from_disk("/kaggle/input/librispeech-val-clean/dataset_val_clean")

def preprocess(batch):
    audio = batch["audio"]["array"]
    batch["input_values"] = processor(audio, sampling_rate=16000, return_tensors="pt").input_values[0]
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

train_dataset = train_dataset.map(preprocess, load_from_cache_file=False, cache_file_name="/kaggle/working/temp.arrow")
valid_dataset = valid_dataset.map(preprocess, load_from_cache_file=False, cache_file_name="/kaggle/working/temp.arrow")

Map:   0%|          | 0/2850 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Map:   0%|          | 0/2703 [00:00<?, ? examples/s]

In [12]:
train_dataset.set_format(type="torch", columns=["input_values", "labels"])
valid_dataset.set_format(type="torch", columns=["input_values", "labels"])

In [13]:
def collate_fn(batch):
    # Pad input_values
    input_values = [item["input_values"] for item in batch]
    input_values = pad_sequence(input_values, batch_first=True)

    # Pad labels
    labels = [torch.tensor(item["labels"]) for item in batch]
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 ignored in loss

    return {"input_values": input_values, "labels": labels}

In [14]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn
)

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Scheduler
scheduler = LambdaLR(optimizer, lr_lambda=lambda step: tri_stage_lr_lambda(
    step,
    total_steps=EPOCHS * len(train_loader),
    warmup_ratio=WARMUP_RATIO,
    hold_ratio=HOLD_RATIO,
    decay_ratio=DECAY_RATIO
))

In [16]:
# ------------------------
# 5️⃣ Training loop (simplified)
# ------------------------
ctc_loss = nn.CTCLoss(blank=processor.tokenizer.pad_token_id, zero_infinity=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Wav2Vec2WithRVQ(
  (wav2vec): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [17]:
model.wav2vec.gradient_checkpointing_enable()

In [18]:
accumulation_steps = GRADIENT_ACCUMULATION_STEPS  # e.g., 8
logging_steps = LOGGING_STEPS  # how often to log
optimizer.zero_grad()

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    # -----------------------
    # TRAINING LOOP
    # -----------------------
    for step, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch}")):
        input_values = batch["input_values"].to(device)   # [B, T]
        labels = batch["labels"].to(device)               # [B, L]

        # Forward pass
        logits = model(input_values)                      # [B, T, vocab]
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

        input_lengths = torch.full(
            size=(logits.size(0),),
            fill_value=logits.size(1),
            dtype=torch.long
        )
        target_lengths = torch.sum(labels != -100, dim=1)

        loss_fn = torch.nn.CTCLoss(
            blank=processor.tokenizer.pad_token_id,
            zero_infinity=True
        )

        loss = loss_fn(
            log_probs.transpose(0, 1),  # [T, B, vocab]
            labels,
            input_lengths,
            target_lengths
        )

        running_loss += loss.item()

        # Normalize for gradient accumulation
        loss = loss / accumulation_steps
        loss.backward()

        # Update optimizer & scheduler every accumulation_steps
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        # Logging every accumulation_steps * logging_steps
        if (step + 1) % (accumulation_steps * logging_steps) == 0:
            avg_loss = running_loss / (accumulation_steps * logging_steps)
            wandb.log({
                "train loss": avg_loss,
                "learning rate": optimizer.param_groups[0]["lr"],
            })
            print(f"Epoch {epoch} Step {step+1} — Avg Train Loss: {avg_loss:.4f}")
            running_loss = 0.0

    # Catch leftover gradients if dataset not divisible by accumulation_steps
    if (step + 1) % accumulation_steps != 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    # -----------------------
    # VALIDATION LOOP
    # -----------------------
    model.eval()
    val_loss = 0.0
    val_predictions, val_references = [], []

    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch}"):
            input_values = batch["input_values"].to(device)
            labels = batch["labels"]

            logits = model(input_values)
            log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

            # Decode predictions
            pred_ids = torch.argmax(logits, dim=-1)
            pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
            ref_str = processor.batch_decode(labels, group_tokens=False)

            val_predictions.extend([p.lower() for p in pred_str])
            val_references.extend([r.lower() for r in ref_str])

            # Compute validation loss
            input_lengths = torch.full(
                size=(logits.size(0),),
                fill_value=logits.size(1),
                dtype=torch.long
            )
            target_lengths = torch.sum(labels != -100, dim=1)

            loss = torch.nn.CTCLoss(
                blank=processor.tokenizer.pad_token_id,
                zero_infinity=True
            )(log_probs.transpose(0, 1), labels, input_lengths, target_lengths)

            val_loss += loss.item()

    val_loss /= len(valid_loader)
    val_wer = wer(hypothesis=val_predictions, reference=val_references)
    print(f"Epoch {epoch} — Validation Loss: {val_loss:.4f}, Validation WER: {val_wer:.4f}")

    # -----------------------
    # TRAIN WER CALCULATION
    # -----------------------
    train_predictions, train_references = [], []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(train_loader, desc=f"Train WER Epoch {epoch}"):
            input_values = batch["input_values"].to(device)
            labels = batch["labels"]

            logits = model(input_values)
            pred_ids = torch.argmax(logits, dim=-1)

            pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
            ref_str = processor.batch_decode(labels, group_tokens=False)

            train_predictions.extend([p.lower() for p in pred_str])
            train_references.extend([r.lower() for r in ref_str])

    train_wer = wer(hypothesis=train_predictions, reference=train_references)
    print(f"Epoch {epoch} — Train WER: {train_wer:.4f}")

    # -----------------------
    # LOGGING
    # -----------------------
    wandb.log({
        "epoch": epoch,
        "train wer": train_wer,
        "validation loss": val_loss,
        "validation wer": val_wer,
    })

print("✅ Fine-tuning with RVQ done!")


Training Epoch 0:   0%|          | 0/357 [00:00<?, ?it/s]/tmp/ipykernel_19/2829265310.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = [torch.tensor(item["labels"]) for item in batch]
Training Epoch 0:  22%|██▏       | 80/357 [03:18<11:44,  2.54s/it]

Epoch 0 Step 80 — Avg Train Loss: 14.7627


Training Epoch 0:  45%|████▍     | 160/357 [06:36<08:14,  2.51s/it]

Epoch 0 Step 160 — Avg Train Loss: 15.1650


Training Epoch 0:  67%|██████▋   | 240/357 [09:57<04:57,  2.55s/it]

...

Epoch 29 Step 320 — Avg Train Loss: 4.1153


Validation Epoch 29: 100%|██████████| 338/338 [02:06<00:00,  2.68it/s]


Epoch 29 — Validation Loss: 5.3880, Validation WER: 1.0000


Train WER Epoch 29: 100%|██████████| 357/357 [02:34<00:00,  2.31it/s]

Epoch 29 — Train WER: 1.0000
✅ Fine-tuning with RVQ done!


In [19]:
torch.save(model.state_dict(), "/kaggle/working/model_with_RVQ.pth")

In [20]:
val_predictions

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'e',
 '',
 '',
 '',
 'e',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'e',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'e',
 '',
 '',
 '',
 

In [21]:
val_references

["he was in a fevered state of mind owing to the blight his wife's action threatened to cast upon his entire future<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>",
 'he would have to pay her the money which she would now regularly demand or there would be trouble it did not matter what he did',
 'hurstwood walked the floor mentally arranging the chief points of his situation<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>',
 'he also thought of his managerial position<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><u